In [ ]:
!nvidia-smi

## Library

In [ ]:
import os
import json
import torch
import torch.nn as n
import torch.nn.functional as F
import argparse
import importlib
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader
from transformers import (AutoTokenizer, 
    AutoConfig, 
    AutoModelForTokenClassification, 
    Trainer, 
    DataCollatorWithPadding
)


## PLM

In [ ]:
TOKENIZER = '/kaggle/input/roberta-large/tokenizer'
PLM = '/kaggle/input/roberta-large/checkpoint/'
MAX_LENGTH = 512
BATCH_SIZE = 4

## Load Datasets

In [ ]:
dir_path = '/kaggle/input/nbme-score-clinical-patient-notes/'

In [ ]:
test_df = pd.read_csv(os.path.join(dir_path, 'test.csv'))
patients_df = pd.read_csv(os.path.join(dir_path, 'patient_notes.csv'))
features_df = pd.read_csv(os.path.join(dir_path, 'features.csv'))

test_df = test_df.merge(features_df, on=['feature_num', 'case_num'], how='left')
test_df = test_df.merge(patients_df, on=['pn_num', 'case_num'], how='left')

In [ ]:
test_df.head()

## Preprocessing Datasets

In [ ]:
feature_text = list(test_df['feature_text'])
pn_history = list(test_df['pn_history'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)

In [ ]:
inputs = [pn_history[i] + tokenizer.sep_token + feature_text[i] for i in range(len(test_df))]
encoded = tokenizer(inputs,
    return_offsets_mapping=True,
    return_token_type_ids=False,
    truncation=True,
)

In [ ]:
input_ids = encoded['input_ids']
attention_mask = encoded['attention_mask']

## Datasets

In [ ]:
class TestDataset(Dataset) :
    def __init__(self, input_ids, attention_mask) :
        super(TestDataset , self).__init__()
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        
    def __len__(self) :
        return len(self.input_ids)

    def __getitem__(self , idx) :
        return {'input_ids' : self.input_ids[idx], 'attention_mask' : self.attention_mask[idx]}

In [ ]:
dataset = TestDataset(input_ids, attention_mask)

## Collator

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=MAX_LENGTH)

## Config & Model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

config = AutoConfig.from_pretrained(PLM)
model = AutoModelForTokenClassification.from_pretrained(PLM, config=config).to(device)

## Dataloader

In [ ]:
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)

## Inference

In [ ]:
offset_mapping = encoded.pop('offset_mapping')

In [ ]:
model.eval()
predictions = []

for data in data_loader :
    
    data = {k:v.to(device) for k,v in data.items()}
    results = model(**data)
    
    logits = results.logits.detach().cpu().numpy()
    prediction_ids = np.argmax(logits, axis=-1)

    preds = [pred for pred in prediction_ids]
    predictions.extend(preds)

## Postprocessing Datasets

In [ ]:
def postprocess(pos_list) :
    idx = 0
    start = 0

    results = []
    while(idx < len(pos_list)) :
        if idx + 1 == len(pos_list) :
            prev_start, prev_end = pos_list[start]
            cur_start, cur_end = pos_list[idx]

            results.append([prev_start, cur_end])
            idx += 1
        else :
            prev_start, prev_end = pos_list[idx]
            cur_start, cur_end = pos_list[idx+1]

            if cur_start == prev_end + 1 or cur_start == prev_end :
                idx += 1
            else :
                span_start = pos_list[start][0]
                span_end = pos_list[idx][1]

                results.append([span_start, span_end])
                start = idx+1
                idx = start
    
    results = [[str(span[0]), str(span[1])] for span in results]
    span_list = [' '.join(span) for span in results]
    return ';'.join(span_list)

In [ ]:
locations = []

for i, pred in enumerate(predictions) :
    offset = offset_mapping[i]
    input_ids = encoded['input_ids'][i]
    
    token_start_index = 1
    token_end_index = input_ids.index(tokenizer.sep_token_id)
    
    span_list = []
    for j in range(token_start_index, token_end_index) :
        if pred[j] == 1 :
            span_list.append(offset[j])
            
    span = '' if len(span_list) == 0 else postprocess(span_list)
    locations.append(span)

In [ ]:
test_df['location'] = locations
test_df = test_df.drop(columns = ['case_num', 'pn_num', 'feature_num', 'feature_text', 'pn_history'])
test_df.head()

In [ ]:
test_df.to_csv('submission.csv', index=False)